<a href="https://colab.research.google.com/github/emmanuelmokel/PPL-practice/blob/main/NestedRHat_TFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -Rf probability
!rm -Rf fun_mc
!rm -Rf inference_gym
!git clone https://github.com/tensorflow/probability.git
!mv probability/spinoffs/fun_mc/fun_mc .
!mv probability/spinoffs/inference_gym/inference_gym .
!pip install tf-nightly tfp-nightly jax jaxlib

!pip install immutabledict

Cloning into 'probability'...
remote: Enumerating objects: 100071, done.
remote: Counting objects: 100% (4855/4855), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 100071 (delta 4734), reused 4745 (delta 4706), pack-reused 95216
Receiving objects: 100% (100071/100071), 136.98 MiB | 9.77 MiB/s, done.
Resolving deltas: 100% (81895/81895), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.8/490.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 42.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3


In [2]:
from matplotlib.pyplot import *

import jax
from jax import random
from jax import numpy as jnp


#!pip install tfp-nightly inference_gym
from inference_gym import using_jax as gym

#from tensorflow_probability.spinoffs import using_jax as fun_mcmc
from fun_mc import using_jax as fun_mcmc

from tensorflow_probability.python.internal import prefer_static as ps
from tensorflow_probability.python.internal import unnest

import tensorflow_probability as _tfp
tfp = _tfp.substrates.jax
tfd = tfp.distributions
tfb = tfp.bijectors

tfp_np = _tfp.substrates.numpy
tfd_np = tfp_np.distributions

import arviz as az
from tensorflow_probability.python.internal.unnest import get_innermost

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
# Working with the 'Banana' Posterior Target Density

target = gym.targets.VectorModel(gym.targets.Banana(),
                                 flatten_sample_transformations = True)
num_dimensions = target.event_shape[0]
# Is the heuristic for the initial value of \epsilon given in the paper used in practice?
init_step_size = 1.

def target_log_prob_fn(x):
   y = target.default_event_space_bijector(x)
   fldj = target.default_event_space_bijector.forward_log_det_jacobian(x)
   return target.unnormalized_log_prob(y) + fldj

offset = 2
def bn_initialize(shape, key = random.PRNGKey(3727709)):
  return 10 * random.normal(key, shape + (num_dimensions,)) + offset

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/internal/backend/jax/ops.py:285: UserWarning: Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return arr.astype(dtype)


In [4]:
# Running default HMC as written in Radford Neal's paper

num_chains = 512
K = 4
num_warmup, num_sampling = 100, 100000
total_samples = num_warmup + num_sampling

kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn, init_step_size, 1)
kernel = tfp.experimental.mcmc.GradientBasedTrajectoryLengthAdaptation(kernel, num_warmup)
kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(kernel, num_warmup,
                                                  target_accept_prob = 0.75,
                                                  reduce_fn = tfp.math.reduce_log_harmonic_mean_exp)

# Initializing each chain (in a super chain) at the same location
initial_state = bn_initialize((K,))
initial_state = jnp.repeat(initial_state, num_chains // K, axis = 0)

In [5]:
%env XLA_PYTHON_CLIENT_ALLOCATOR = platform
result = tfp.mcmc.sample_chain(total_samples, initial_state,
                               kernel = kernel, seed = random.PRNGKey(1954))

env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '


In [6]:
# This object is of type tfp.mcmc.CheckpointableStatesAndTrace
# It contains attributes all_states, trace, and final_kernel_results
# https://www.tensorflow.org/probability/api_docs/python/tfp/mcmc/CheckpointableStatesAndTrace

result

StatesAndTrace(
  all_states=Array([[[-14.236393  ,   3.871798  ],
            [-13.070528  ,   4.7461495 ],
            [-12.489271  ,   4.8525343 ],
            ...,
            [  0.03275418,  -3.6777086 ],
            [ -0.666454  ,  -2.3977256 ],
            [ -1.2655139 ,  -4.4068384 ]],
    
           [[-14.236393  ,   3.871798  ],
            [-13.070528  ,   4.7461495 ],
            [-12.489271  ,   4.8525343 ],
            ...,
            [  0.03275418,  -3.6777086 ],
            [ -0.666454  ,  -2.3977256 ],
            [ -1.2655139 ,  -4.4068384 ]],
    
           [[-14.236393  ,   3.871798  ],
            [-13.070528  ,   4.7461495 ],
            [-12.489271  ,   4.8525343 ],
            ...,
            [  0.03275418,  -3.6777086 ],
            [ -0.666454  ,  -2.3977256 ],
            [ -1.2655139 ,  -4.4068384 ]],
    
           ...,
    
           [[  4.145871  ,  -1.2746216 ],
            [ -0.3929863 ,  -3.4548879 ],
            [ -0.90132916,  -2.725726  ],
   

In [7]:
import tensorflow as tf
tf.executing_eagerly()

True

In [8]:
from tensorflow.python.ops.linalg_ops import matrix_determinant
# This is the implementation from the rhat_locker.ipynb

"""
# Inquire about _axis_size as a function, and what it is meant to do

#def _axis_size(x, axis = None):
#  return

# What is Pavel's example? How can this be rewritten without tf (maybe JAX)?
# What is the purpose of removing TF
def _reduce_variance(x, axis= None, biased = True, keepdims = False):
  with tf.name_scope('reduce_variance'):
    x = tf.convert_to_tensor(x, name = 'x')
    mean = tf.reduce_mean(x, axis=axis, keepdims=True)
    biased_var = tf.reduce_mean(
        tf.math.squared_difference(x, mean), axis = axis, keepdims = keepdims)
    if biased:
      return biased_var
    n = _axis_size(x, axis)
    return (n / (n-1.))*biased_var


def nested_rhat(result_state, num_super_chains):
  used_samples = result_state.shape[0]
  num_sub_chains = result_state.shape[1]
  num_dimensions = result_state.shape[2]

  chain_states = result_state.reshape(used_samples, -1, num_sub_chains,
                                      num_dimensions)
  state = tf.convert_to_tensor(chain_states, name = 'state')
  mean_chain = tf.reduce_mean(state, axis = 0)
  mean_super_chain = tf.reduce_mean(state, axis = [0, 2])
  variance_chain = _reduce_variance(state, axis = 0, biased = True)
  variance_super_chain = _reduce_variance(state, axis = 0, biased = True) \
     + tf.reduce_mean(variance_chain, axis = -1)

  W = tf.reduce_mean(variance_super_chain, axis = 0)
  B = _reduce_variance(mean_super_chain, axis = 0, biased = True)

  return tf.sqrt((W+B) / W)

"""

"\n# Inquire about _axis_size as a function, and what it is meant to do\n\n#def _axis_size(x, axis = None):\n#  return\n\n# What is Pavel's example? How can this be rewritten without tf (maybe JAX)?\n# What is the purpose of removing TF\ndef _reduce_variance(x, axis= None, biased = True, keepdims = False):\n  with tf.name_scope('reduce_variance'):\n    x = tf.convert_to_tensor(x, name = 'x')\n    mean = tf.reduce_mean(x, axis=axis, keepdims=True)\n    biased_var = tf.reduce_mean(\n        tf.math.squared_difference(x, mean), axis = axis, keepdims = keepdims)\n    if biased:\n      return biased_var\n    n = _axis_size(x, axis)\n    return (n / (n-1.))*biased_var\n\n\ndef nested_rhat(result_state, num_super_chains):\n  used_samples = result_state.shape[0]\n  num_sub_chains = result_state.shape[1]\n  num_dimensions = result_state.shape[2]\n\n  chain_states = result_state.reshape(used_samples, -1, num_sub_chains,\n                                      num_dimensions)\n  state = tf.convert

In [9]:
# Define nested Rhat for one parameter.
# This assumes the indexed parameter is a scalar, hence the result_state
# tensor needs to be formatted accordingly.
# TODO: deprecate state_is_list argument
def nested_rhat_1dim(result_state, num_super_chains, index_param,
                     num_samples, warmup_length = 0,
                     rank_normalize = False):
                     # state_is_list = False, vector_index = None):

  # The below code is DEPRECATED
  # if state_is_list:
  #   if vector_index is not None:
  #     state_param = result_state[index_param][
  #                 warmup_length:(warmup_length + num_samples), :, vector_index]
  #   else:
  #     state_param = result_state[index_param][
  #                       warmup_length:(warmup_length + num_samples), :]
  # else:
  #   state_param = result_state[warmup_length:(warmup_length + num_samples),
  #                              :, index_param]

  state_param = result_state[warmup_length:(warmup_length + num_samples),
                             :, index_param]

  num_samples = state_param.shape[0]
  num_chains = state_param.shape[1]
  num_sub_chains = num_chains // num_super_chains
  total_samples = num_samples * num_chains

  if (rank_normalize):
    state_param_flat = jnp.reshape(state_param, (total_samples, ))
    temp = state_param_flat.argsort()
    ranks = temp.argsort() + 1
    z = norm.ppf((ranks - 3 / 8) / (total_samples + 1 / 4))
    state_parm = jnp.reshape(z, (num_samples, num_chains))

  state_param = state_param.reshape(num_samples, -1, num_sub_chains, 1)

  mean_chain = jnp.mean(state_param, axis = (0, 3))
  between_chain_var = jnp.var(mean_chain, axis = 1, ddof = 1)
  if (num_samples == 1):
    mean_within_chain_var = 0
  else:
    within_chain_var = jnp.var(state_param, axis = (0, 3), ddof = 1)
    mean_within_chain_var = jnp.mean(within_chain_var, axis = 1)

  total_chain_var = between_chain_var + mean_within_chain_var

  mean_super_chain = jnp.mean(state_param, axis = (0, 2, 3))
  between_super_chain_var = jnp.var(mean_super_chain, ddof = 1)

  return jnp.sqrt(1 + between_super_chain_var / jnp.mean(total_chain_var)),\
    between_super_chain_var, jnp.mean(total_chain_var)


def nested_rhat(result_state, num_super_chains, index_param,
                num_samples, warmup_length = 0, rank_normalize = False):
  nRhat = jnp.array([])
  B = jnp.array([])
  W = jnp.array([])
  for i in range(0, index_param.shape[0]):
    nRhat_local, B_local, W_local = nested_rhat_1dim(result_state,
                    num_super_chains, index_param[i], num_samples,
                    warmup_length, rank_normalize)

    nRhat = jnp.append(nRhat, nRhat_local)
    B = jnp.append(B, B_local)
    W = jnp.append(W, W_local)

  return nRhat, B, W

## Using ChEES-HMC in order to get the most out of a GPU

In [10]:
# Select HMC Variant from options chees, snaper, or malt

def construct_kernel(target_log_prob_fn, init_step_size, num_warmup, transition = 'chees'):
  if transition == 'chees':
    # Using gradient-based trajectory length adaptation and dual averaging defines ChEES HMC
    kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn, init_step_size, 1)
    kernel = tfp.experimental.mcmc.GradientBasedTrajectoryLengthAdaptation(kernel, num_warmup)
    kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
        kernel, num_warmup, target_accept_prob = 0.75,
        reduce_fn = tfp.math.reduce_log_harmonic_mean_exp)

  elif transition == 'snaper':
    # TO BE IMPLEMENTED
    kernel = 'snaper'

  elif transition == 'malt':
    # TO BE IMPLEMENTED
    kernel = 'malt'

  else:
    raise Exception("Not a valid transition kernel. Try one of ['chees', 'snaper', 'malt']")

  return kernel

In [14]:
jnp.array([0, 1]).shape[0]

2

In [12]:
def run_fits(num_seed, total_samples, initialize, kernel, num_super_chains,
             index_param, num_samples, num_warmup, rank_normalize = False):

  num_parameters = index_param.shape[0]

  Rhat_list = jnp.zeros((num_seed, num_parameters))
  nRhat_list = jnp.zeros((num_seed, num_parameters))
  B_list = jnp.zeros((num_seed, num_parameters))
  W_list = jnp.zeros((num_seed, num_parameters))
  mc_mean_list = jnp.zeros((num_seed, num_parameters))

  i = 0
  for seed in jax.random.split(jax.random.PRNGKey(1), num_seed):
    initial_state = initialize((num_super_chains,), key = seed + 1954)

    initial_state = jnp.repeat(initial_state, num_chains // num_super_chains,
                              axis = 0)

    result = tfp.mcmc.sample_chain(total_samples, initial_state, kernel = kernel, seed = seed)

    # Implementation of original R-Hat from Gelman and Rubin (1992)
    Rhat_list = Rhat_list.at[i, :].set(tfp.mcmc.potential_scale_reduction(result.all_states[num_warmup:(num_warmup + num_samples), :, index_param]))

    nRhat_local, B_local, W_local = nested_rhat(result.all_states,
                                                num_super_chains = num_super_chains,
                                                index_param = index_param,
                                                num_samples = num_samples,
                                                warmup_length = num_warmup,
                                                rank_normalize = rank_normalize)
    nRhat_list = nRhat_list.at[i, :].set(nRhat_local)
    B_list = B_list.at[i, :].set(B_local)
    W_list = W_list.at[i, :].set(W_local)

    mc_mean_list = mc_mean_list.at[i, :].set(jnp.mean(result.all_states[num_warmup + 1, :, index_param], axis = 1))
    i += 1

  return Rhat_list, nRhat_list, B_list, W_list, mc_mean_list


In [13]:
# Adaptive warmup scheme to produce MCMC samples
def forge_chain(kernel_cold, kernel_warm, initial_state, num_super_chains,
                num_warmup_array, num_samples, target_rhat, max_num_steps,
                index_param, seed, num_nRhat_comp = 1, rank_normalize = False,
                alpha_quantile = 1, mean_benchmark = None, var_benchmark = None):
  warmup_is_acceptable = False
  window_iteration = 0
  current_state = initial_state
  kernel_args = None

  squared_err_list = jnp.array([])
  nrhat_list = jnp.array([])

  while(not warmup_is_acceptable and window_iteration < max_num_steps):

    # Runs MCMC with a warmup window
    result_cold, trace, kernel_args = tfp.mcmc.sample_chain(
        num_results = num_warmup_array[window_iteration],
        current_state = current_state,
        kernel = kernel_cold,
        previous_kernel_results = kernel_args,
        trace_fn = lambda _, pkr: unnest.get_innermost(pkr, 'step_size'),
        return_final_kernel_results = True,
        seed = seed + window_iteration)

    current_state = result_cold[-1]

    # Generate candidate samples
    result_warm, trace = tfp.mcmc.sample_chain(
        num_results = num_samples*num_nRhat_comp,
        current_state = current_state,
        kernel = kernel_warm,
        trace_fn = lambda _, pkr: unnest.get_innermost(pkr, 'step_size'),
        previous_kernel_results = kernel_args,

        # Why are we just adding numbers to seeds like this? I get it is to randomize but is there a more principled way?
        seed = seed + 999999)

    # Check if candidate samples are acceptable
    nRhat = jnp.zeros((index_param.shape[0], num_nRhat_comp))
    for i in range(0, num_nRhat_comp):
      nR, _B, _W = nested_rhat(result_warm[i:((i+1)*num_samples)],
                                        num_super_chains = num_super_chains,
                                        index_param = index_param,
                                        num_samples = num_samples,
                                        rank_normalize = rank_normalize)
      nRhat = nRhat.at[:, i].set(nR)

    nRhat_quantile = jnp.quantile(jnp.mean(nRhat, axis = 1), alpha_quantile,
                                 interpolation = "nearest")
    print("nRhat_quantile: (", alpha_quantile, ")", nRhat_quantile)

    if mean_benchmark is not None:
      mc_mean = jnp.mean(result_warm[0, :, index_param], axis = 1)
      squared_err = jnp.square(mc_mean - mean_benchmark[index_param])\
      / var_benchmark[index_param]

      squared_err_list = jnp.append(squared_err_list, squared_err)
      nrhat_list = jnp.append(nrhat_list, jnp.mean(nRhat, axis = 1))

    if (nRhat_quantile < target_rhat): warmup_is_acceptable = True

    window_iteration += 1

  return result_warm, window_iteration, squared_err_list, nrhat_list

In [14]:
def run_forge_chain(num_seed, kernel_cold, kernel_warm, initialize, num_super_chains,
                    num_warmup, num_samples, target_rhat, max_num_steps,
                    index_param, num_nRhat_comp = 1, rank_normalize = False,
                    alpha_quantile = 1, mean_benchmark = None,
                    var_benchmark = None, initial_seed = 1):
  mc_mean_list = jnp.zeros((num_seed, index_param.shape[0]))
  warmup_length = jnp.zeros(num_seed)

  squared_err_list_all = jnp.array([])
  nrhat_list_all = jnp.array([])

  i = 0
  for seed in jax.random.split(jax.random.PRNGKey(initial_seed), num_seed):
    print("NEW SEED")
    initial_state = initialize((num_super_chains,), key = seed + 1954)
    initial_state = jnp.repeat(initial_state, num_chains // num_super_chains,
                              axis = 0)

    result, window_iteration, \
    squared_err_list, nrhat_list = forge_chain(kernel_cold, kernel_warm,
                                               initial_state, num_super_chains,
                                               num_warmup, num_samples,
                                               target_rhat, max_num_steps,
                                               index_param, seed,
                                               num_nRhat_comp, rank_normalize,
                                               alpha_quantile, mean_benchmark,
                                               var_benchmark)

    warmup_length = warmup_length.at[i].set(sum(num_warmup[:window_iteration]))
    mc_mean_list = mc_mean_list.at[i, :].set(jnp.mean(result[0, :, index_param],
                                 axis = 1))

    squared_err_list_all = jnp.append(squared_err_list_all, squared_err_list)
    nrhat_list_all = jnp.append(nrhat_list_all, nrhat_list)

    i += 1
  return mc_mean_list, warmup_length, squared_err_list_all, nrhat_list_all

In [15]:
num_warmup, num_samples = 1000, 5
total_samples = num_warmup + num_samples + 1

kernel = construct_kernel(target_log_prob_fn = target_log_prob_fn,
                          init_step_size = init_step_size, num_warmup = num_warmup,
                          transition = 'chees')
index_param = jnp.array([0, 1])
num_seed = 30
rank_normalize = False

In [16]:
Rhat_list, nRhat_list, B_list, W_list, mc_mean_list = run_fits(
           num_seed = num_seed, total_samples = total_samples,
           initialize = bn_initialize, kernel = kernel,
           num_super_chains = K, index_param = index_param,
           num_samples = num_samples, num_warmup = num_warmup,
           rank_normalize = rank_normalize)

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback tha

In [22]:
# What is the use of a second dimension for computing rHat?
nRhat_list.shape
jnp.var(nRhat_list[:,1])

Array(1.0652867e-06, dtype=float32)

In [31]:
mc_mean_list

Array([[ 0.67108625,  0.08250239],
       [-0.23767622,  0.07473338],
       [ 0.11974052,  0.05402053],
       [-0.0779959 , -0.29601008],
       [-0.3021367 ,  0.40538344],
       [ 0.5211482 , -0.16167125],
       [ 0.58794963,  0.0190184 ],
       [-0.95380914,  0.40924037],
       [ 0.60984075, -0.3701806 ],
       [-0.5059649 ,  0.35890698],
       [ 0.20344737, -0.23870113],
       [ 0.05978912, -0.20745602],
       [ 0.11211918, -0.22980669],
       [-0.2892716 ,  0.0647719 ],
       [ 0.12771514, -0.18332335],
       [-0.06098558, -0.3125804 ],
       [-0.01340416, -0.12903082],
       [ 0.262689  , -0.16189626],
       [ 0.5807346 ,  0.13833123],
       [ 0.5393245 ,  0.14760461],
       [ 0.09519129, -0.0539025 ],
       [ 0.23255001, -0.04695915],
       [ 0.17679004, -0.36940455],
       [-0.17988108,  0.08165627],
       [-0.05505823,  0.07657137],
       [ 0.2515199 , -0.3048549 ],
       [-0.3819338 ,  0.08875138],
       [ 0.15690205, -0.46691388],
       [ 0.1646577 ,

In [32]:
# Parellelizing the calculations of nRhat instead of sequential running
# This involves getting rid of the loop inside of the code for the regular run_fits(...)
# Does this change how the seeds are calculated at all?
# Removing the num_seed argument if this won't be run sequentially

def run_fits_parallel(iterations, total_samples, initialize, kernel, num_super_chains,
             index_param, num_samples, num_warmup, rank_normalize = False):

  num_parameters = index_param.shape[0]

  Rhat_list = jnp.zeros((num_seed, num_parameters))
  nRhat_list = jnp.zeros((num_seed, num_parameters))
  B_list = jnp.zeros((num_seed, num_parameters))
  W_list = jnp.zeros((num_seed, num_parameters))
  mc_mean_list = jnp.zeros((num_seed, num_parameters))

  # Should this be a more explicit parameter into the method for reproducibility purposes?
  seed = jax.random.PRNGKey(1)
  initial_state = initialize((num_super_chains*iterations,), key = seed + 1954)

  initial_state = jnp.repeat(initial_state, num_chains*iterations // num_super_chains,
                              axis = 0)

  result = tfp.mcmc.sample_chain(total_samples, initial_state, kernel = kernel, seed = seed)

  # Loop over K superchains at a time in order to calculate nRhat statistics
  # Investigate how JAX and looping may be impacting the speed of this loop

  for i in range(iterations):
    # Selecting the subset that corresponds to K superchains of size M
    current_result = result.all_states[num_warmup:(num_warmup + num_samples), (i*num_chains):((i+1)*num_chains), index_param]
    Rhat_list = Rhat_list.at[i, :].set(tfp.mcmc.potential_scale_reduction)

    # Implementation of original R-Hat from Gelman and Rubin (1992)

    Rhat_list = Rhat_list.at[i, :].set(tfp.mcmc.potential_scale_reduction(current_result))

    nRhat_local, B_local, W_local = nested_rhat(current_result,
                                                num_super_chains = num_super_chains,
                                                index_param = index_param,
                                                num_samples = num_samples,
                                                warmup_length = num_warmup,
                                                rank_normalize = rank_normalize)
    nRhat_list = nRhat_list.at[i, :].set(nRhat_local)
    B_list = B_list.at[i, :].set(B_local)
    W_list = W_list.at[i, :].set(W_local)

    mc_mean_list = mc_mean_list.at[i, :].set(jnp.mean(result.all_states[num_warmup + 1, (i*num_chains):((i+1)*num_chains), index_param], axis = 1))
    i += 1

  return Rhat_list, nRhat_list, B_list, W_list, mc_mean_list

In [36]:
# This yields an error; is this just because Colab doesn't have the strongest GPUs?
run_fits_parallel(
           iterations = num_seed, total_samples = total_samples,
           initialize = bn_initialize, kernel = kernel,
           num_super_chains = K, index_param = index_param,
           num_samples = num_samples, num_warmup = num_warmup,
           rank_normalize = rank_normalize)

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '


XlaRuntimeError: ignored

In [ ]:
def adaptive_initialize(target_log_prob_fn, init_step_size, num_warmup, warmup_window):
    kernel_cold = construct_kernel(target_log_prob_fn = target_log_prob_fn,
                                   init_step_size = init_step_size,
                                   num_warmup = num_warmup)
    kernel_warm = construct_kernel(target_log_prob_fn = target_log_prob_fn,
                                   init_step_size = init_step_size,
                                   num_warmup = num_warmup)
    window_array = jnp.append(jnp.repeat(10, 10),
                              jnp.repeat(warmup_window,
                                         num_warmup // warmup_window - 1))
    nRhat_upper = 1.05

    try:
      mean_est = target.sample_transformations['identity'].ground_truth_mean
    except:
      print('no ground truth mean')
      mean_est = (result.all_states[num_warmup:, :]).mean(0).mean(0)
    try:
      var_est = target.sample_transformations['identity'].ground_truth_standard_deviation**2
    except:
      print('no ground truth std dev')
      var_est = ((result.all_states[num_warmup:, :]**2).mean(0).mean(0) -
                mean_est**2)

    return kernel_cold, kernel_warm, window_array, nRhat_upper, mean_est, var_est

In [ ]:
# This will need to be written in a way such that it can be integrated into the function below
def hmc_bootstrap():
  return

In [ ]:
"""
We will now focus on variance measurement of nested Rhat, with users having the
possibility to choose several parameters such as the number of superchains (K),
number of markov chains in each group of superchains (M), and number of draws in
the sampling phase (N). We will measure this in both a brute force manner, as
well as coming up with a notion of bootstrapping. We also can decide to run this
variance estimate across all chains sequentially, or in parallel. Having an
adaptive warmup length, or a fixed numbers of warmup iterations is also a
potential parameter

Using an initial seed value of 1

I need to think a bit about parallelization as well as bootstrapping

"""

# There's 8 possibilities here that come from 2^[boostrap, adaptive, parallel]
def variance_estimate(target_log_prob_fn, index_param, num_seed = 10, num_warmup = 1000, num_samples = 5,
                      num_super_chains = 4, initialize = bn_initialize, num_chains = 128,
                      bootstrap = False, adaptive = False, parallel = False,
                      warmup_window = 100, initial_seed = 1):

  if adaptive:

    kernel_cold, kernel_warm, window_array, \
    nRhat_upper, mean_est, var_est = adaptive_initialize(target_log_prob_fn,
                                                         init_step_size,
                                                         num_warmup,
                                                         warmup_window)

    if bootstrap:
      if parallel:
        # Adpative, Bootstrapped, and Parallel

        return

      else:
        # Adaptive, Bootstrapped, and Sequential
        return

    else:
      if parallel:
        # Adpative, Brute-Force, and Parallel
        return

      else:
        # Adaptive, Brute-Force, and Sequential
        mc_mean_list, warmup_length, squared_error_list, \
        nRhat_list = run_forge_chain(num_seed = num_seed,
                                     kernel_cold = kernel_cold,
                                     kernel_warm = kernel_warm,
                                     initialize = initialize,
                                     num_super_chains = num_super_chains,
                                     num_warmup = window_array,
                                     num_samples = num_samples,
                                     target_rhat = nRhat_upper,
                                     max_num_steps = window_array.shape[0],
                                     index_param = index_param,
                                     rank_normalize = False,
                                     alpha_quantile = 1.,
                                     mean_benchmark = mean_est,
                                     var_benchmark = var_est)

  else:

    kernel = construct_kernel(target_log_prob_fn = target_log_prob_fn,
                              init_step_size = init_step_size,
                              num_warmup = num_warmup)
    if bootstrap:

      if parallel:
        # Fixed, Bootstrapped, and Parallel
        return

      else:
        # Fixed, Bootstrapped, and Sequential
        return

    else:
      if parallel:
        # Fixed, Brute-Force, and Parallel
        return

      else:
        # Fixed, Brute-Force, and Sequential
        # Adress apparent discrepancy in definition of total_samples
        Rhat_list, nRhat_list, B_list, W_list, mc_mean_list = run_fits(
            num_seed = num_seed, total_samples = num_warmup + num_samples + 1,
            initialize = initialize, kernel = kernel,
            num_super_chains = num_super_chains,
            index_param = index_param, num_samples = num_samples,
            num_warmup = num_warmup)

  return nRhat_list

In [ ]:
# After calculating the variance of nRhat across several runs we can create visualization
# Must think about what is a helpful type of visualization in this situation

estimates = variance_estimate(target_log_prob_fn, index_param)

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback tha

In [ ]:
es

Array([[1.0007871, 1.0007992],
       [1.0009911, 1.0010496],
       [1.0003695, 1.0017493],
       [1.0003909, 1.0003959],
       [1.0007714, 1.0024724],
       [1.0007972, 1.0030594],
       [1.0016471, 1.0036765],
       [1.000586 , 1.0000635],
       [1.0010842, 1.0008926],
       [1.0008079, 1.0006552]], dtype=float32)